# Ecuación de difusión en 1D por el método de Fourier

## Importación de bibliotecas

In [17]:
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display
 
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt

## Variables globales

In [2]:
D = 0.5
Lx = 10.0
A = 2.0
x0 = 5.0
l = 1.5

## Funciones

In [19]:
def funcion(x,n):
    """
    Función a integrar para encontrar los bn.
    Parámetros:
        x: variable espacial
        n: indica el término a calcular
    Salida:
        función evaluada en x,n
    """
    funcion = A*np.exp(-(x-x0)**2/l)*np.sin(n*np.pi*x/Lx)
    return funcion

In [20]:
def calcularB(n):
    """
    Calcula el valor del bn
    Parámetro:
        n: indica el término a calcular
    Salida:
        b: valor de bn
    """
    integral, error = integrate.quad(funcion,0,Lx,args=(n))
    b = 2*integral/Lx
    return b

In [24]:
def AproximarRho(x,t,numTerminos):
    """
    Calcular el valor aproximado de rho en el punto (x,t)
    Parámetros:
        x: posición(es) en el eje x (variable espacial)
        y: posición(es) en el eje t (variable temporal)
        numTerminos: cantidad de términos a utilizar en la aproximación
    Salidas:
        valorAproxRho: valor(es) de rho en el punto (x,t)
    """
    # Se inicializa el valorAproxRho
    valorAproxRho = 0

    # Se realiza la sumatoria hasta alcanzar el número de términos solicitado
    for n in range(1, numTerminos+1):
        b = calcularB(n)
        valorAproxRho += b*np.exp(-((n*np.pi/Lx)**2)*t/D)*np.sin(n*np.pi*x/Lx)
    return valorAproxRho

In [25]:
def GraficarRho(puntosMalla,numTerminos):
    """
    Calcula el valor aproximado de rho en todos los puntos de la malla y lo grafica
    Parámetros:
        puntosMalla: cantidad de puntos en cada eje de la malla
        numTerminos: cantidad de términos a utilizar en la aproximación
    Salida:
        no aplica
    """
    # Se define la malla
    x = np.linspace(0,Lx,puntosMalla)
    t = np.linspace(0,Lx,puntosMalla)
    X,T = np.meshgrid(x,t)
    Rho = AproximarRho(X, T, numTerminos)
    
    #Graficación
    plt.figure(figsize=(10,6)) 
    ax = plt.axes(projection='3d')
    ax.set_xlabel('x')
    ax.set_ylabel('t')
    ax.set_zlabel('rho')
    ax.plot_surface(X, T, Rho, rstride=1, cstride=1, cmap='cividis', edgecolor='none')
    ax.set_title('Aproximacion Rho')
    plt.show()
    
    return

## Main

In [26]:
y=interactive(GraficarRho, {'manual': True}, puntosMalla=widgets.IntSlider(min=10.0, max=50.0, step=5.0, value=30.0, description='Puntos Malla:'),
              numTerminos=widgets.IntSlider(min=10.0, max=150.0, step=25.0, value=25.0, description='# Términos:'))
display(y)

interactive(children=(IntSlider(value=30, description='Puntos Malla:', max=50, min=10, step=5), IntSlider(valu…